In [ ]:
#Instalando as bibliotecas necessárias
#%pip install pandas


In [1]:
## Importando as bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
## Carregando os arquivos para o ambiente
def carregar_dados():
    df2024 = pd.read_csv(r'C:\Repos\tcc-bruna\fonte.csv')
    df2019 = pd.read_excel(r'C:\Repos\tcc-bruna\verificadoativos07-05-2019abr-2019.xlsx')
    return df2024, df2019

df2024, df2019 = carregar_dados()

C:\Users\freda\AppData\Local\Temp\ipykernel_3088\1237448594.py:3: DtypeWarning: Columns (0,1,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df2024 = pd.read_csv(r'C:\Repos\tcc-bruna\fonte.csv')


In [3]:
## Preparando os dados para iniciar a analise

def preparar_dados(df, colunas_desnecessarias, setores_ignorados):
    ## Remover as observações que não possuem a variável 'SETOR'
    df = df.dropna(subset=['SETOR'])
    
    ## Remover as variáveis que não serão utilizadas
    colunas_a_remover = [col for col in colunas_desnecessarias if col in df.columns]
    df = df.drop(columns=colunas_a_remover)   

    ## Vamos remover todas as observações que não possuem a substring ''BIBLIOTECA'' na coluna 'SETOR'
    df = df[df['SETOR'].str.contains('BIBLIOTECA')]
    
    ## Remover as observações que não serão analisadas
    df = df[~df['SETOR'].isin(setores_ignorados)]
    
    return df

colunas_desnecessarias = ['VINCULO', 'REF_CARGO_BAS', 'GRUPO', 'REF_CARGO_COM', 'ESCOL_CARGO_COMISSAO', 'JORNADA', 'ORGAO_EXT', 'RACA', 'DEFICIENTE']

setores_ignorados = [
    'BIBLIOTECA MUNICIPAL MARIO DE ANDRADE',
    'BIBLIOTECA PUBLICA MUNICIPAL LOUIS BRAILLE',
    'BIBLIOTECA PUBLICA MUNICIPAL SERGIO MILLIET',
    'BIBLIOTECA JOAO CABRAL DE MELO NETO - CEU VILA CUR',
    'BIBLIOTECA JORNALISTA ROBERTO MARINHO - CEU BUTANT',
    'BIBLIOTECA RACHEL DE QUEIROZ - CEU ALVARENGA'
]

df2024 = preparar_dados(df2024, colunas_desnecessarias, setores_ignorados)
df2019 = preparar_dados(df2019, colunas_desnecessarias, setores_ignorados)


In [4]:
## Visualizar o resultado da preparação dos dados

def viualizar_dados(df):
    ## Visualizar as primeiras observações de df2024
    df.info()
    df.head()
    
viualizar_dados(df2024)
viualizar_dados(df2019)

<class 'pandas.core.frame.DataFrame'>
Index: 339 entries, 115 to 127354
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   REGISTRO            339 non-null    object
 1   NOME                339 non-null    object
 2   CARGO_BASICO        339 non-null    object
 3   SEGMENTO            107 non-null    object
 4   SUBGRUPO            339 non-null    object
 5   ESCOL_CARGO_BASICO  339 non-null    object
 6   CARGO_COMISSAO      68 non-null     object
 7   DATA_INICIO_EXERC   339 non-null    object
 8   REL_JUR_ADM         339 non-null    object
 9   SECRET_SUBPREF      339 non-null    object
 10  SETOR               339 non-null    object
 11  SEXO                339 non-null    object
 12  ANO_NASCIMENTO      339 non-null    object
dtypes: object(13)
memory usage: 37.1+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 363 entries, 151 to 121748
Data columns (total 13 columns):
 #   Column              Non

In [9]:
## Nessa função, vamos analisar a quantidade de servidores e servidoras que poderão se aposentar em 2024. Para isso, vamos considerar as condições da regra de transição (categoria pedágio): Para ambos os sexos, ter mais de 20 anos de serviço público. Para homens, ter mais de 60 anos de idade. Para mulheres, ter mais de 57 anos de idade.

def analise_aposentadorias_para_ano(df2024, df2019 ,projecao_ano):

    # Iniciaremos resolvendo a inconsistência de datas de início de exercício dos datasets. Devido a reestruturação de carreira que aconteceu em 2022, os servidores que optaram por mudar a carreira, tiveram a data de início de exercício alterada para a data de início da nova carreira. Para resolver essa inconsistência, vamos comparar as datas de início de exercício dos servidores que estão presentes nos dois datasets e atualizar a data de início de exercício de 2024 para a data de início de exercício de 2019, caso seja diferente a fim de calcularmos o tempo de serviço público de forma correta.

    df2024['DATA_INICIO_EXERC'] = pd.to_datetime(df2024['DATA_INICIO_EXERC'], errors='coerce')
    df2019['DATA_INICIO_EXERC'] = pd.to_datetime(df2019['DATA_INICIO_EXERC'], errors='coerce')
    
    # Inicializar a coluna 'CONSISTENTE' com True
    df2024['CONSISTENTE'] = True
    
    # Iterar sobre as linhas do df2024
    for idx, row in df2024.iterrows():
        nome = row['NOME']
        data_inicio_exerc_2024 = row['DATA_INICIO_EXERC']
        
        # Procurar o mesmo servidor no df2019
        if nome in df2019['NOME'].values:
            data_inicio_exerc_2019 = df2019[df2019['NOME'] == nome]['DATA_INICIO_EXERC'].values[0]
            
            # Comparar as datas e atualizar se necessário
            if pd.notna(data_inicio_exerc_2019) and (data_inicio_exerc_2024 != data_inicio_exerc_2019):
                df2024.at[idx, 'DATA_INICIO_EXERC'] = data_inicio_exerc_2019
                df2024.at[idx, 'CONSISTENTE'] = False
    
    ## Resolvida as inconsistências, vamos calcular o tempo de serviço público de cada servidor, e sua idade no ano que será feita a projeção.

    # Calcular a idade
    df2024[f'IDADE_{projecao_ano}'] = projecao_ano - (df2024['ANO_NASCIMENTO']).astype(int)

    # Calcular o tempo de serviço público até 2024
    df2024[f'TEMPO_SERVICO_PUBLICO_{projecao_ano}'] = projecao_ano - df2024['DATA_INICIO_EXERC'].dt.year

    # Criar a variável 'APTO_APOSENTADORIA'

    # Homens: idade >= 60 e tempo de serviço público >= 20
    # Mulheres: idade >= 57 e tempo de serviço público >= 20
    df2024[f'APTO_APOSENTADORIA_{projecao_ano}'] = (df2024[f'IDADE_{projecao_ano}'] >= 60) & (df2024[f'TEMPO_SERVICO_PUBLICO_{projecao_ano}'] >= 20) & (df2024['SEXO'] == 'M') | (df2024[f'IDADE_{projecao_ano}'] >= 57) & (df2024[f'TEMPO_SERVICO_PUBLICO_{projecao_ano}'] >= 20) & (df2024['SEXO'] == 'F')

    
    # Retornar o DataFrame atualizado
    return df2024

df_aposen = analise_aposentadorias_para_ano(df2024, df2019, 2024)
df_aposen = analise_aposentadorias_para_ano(df2024, df2019, 2029)
df_aposen.head()

,REGISTRO,NOME,CARGO_BASICO,SEGMENTO,SUBGRUPO,ESCOL_CARGO_BASICO,CARGO_COMISSAO,DATA_INICIO_EXERC,REL_JUR_ADM,SECRET_SUBPREF,SETOR,SEXO,ANO_NASCIMENTO,CONSISTENTE,IDADE_2029,TEMPO_SERVICO_PUBLICO_2029,APTO_APOSENTADORIA_2029,IDADE_2024,TEMPO_SERVICO_PUBLICO_2024,APTO_APOSENTADORIA_2024
115,1949136,NEUSA PRADO LIMA,ANALISTA DE INFORMACOES CULTURA E DESPORTO (NQ...,BIBLIOTECONOMIA,SUPERIOR,SUPERIOR COMPLETO,NaN,1983-03-23,EFETIVO,SECRETARIA MUNICIPAL DE CULTURA,BIBLIOTECA PUBLICA MUNICIPAL HANS CHRISTIAN ANDER,F,1949,True,80,46,True,75,41,True
202,3133290,LIDIA ZILNAY DOURADO GOMES,ASSISTENTE ADMINISTRATIVO DE GESTAO,NaN,MEDIO,ENSINO MEDIO - COMPLETO,NaN,1977-02-24,ADMITIDO,SECRETARIA MUNICIPAL DE CULTURA,BIBLIOTECA PUBLICA MUNICIPAL JOSE PAULO PAES,F,1955,True,74,52,True,69,47,True
244,3159566,CELIA DINIZ,ANALISTA DE INFORMACOES CULTURA E DESPORTO (NQ...,BIBLIOTECONOMIA,SUPERIOR,SUPERIOR COMPLETO,ASSESSOR I,1989-07-17,EFETIVO,SECRETARIA MUNICIPAL DE CULTURA,SUPERVISAO DE BIBLIOTECAS,F,1955,True,74,40,True,69,35,True
255,3167623,ROSA APARECIDA BIN OLIVEIRA,ASSISTENTE ADMINISTRATIVO DE GESTAO NIVEL II,NaN,MEDIO,ENSINO MEDIO - COMPLETO,NaN,1990-06-18,EFETIVO,SECRETARIA MUNICIPAL DE CULTURA,BIBLIOTECA PUBLICA MUNICIPAL AFFONSO TAUNAY,F,1957,True,72,39,True,67,34,True
360,4762266,JOSE AUGUSTO SECCO,ASSISTENTE DE SUPORTE OPERACIONAL NIVEL III,NaN,BASICO,FUND II - COMPLETO,NaN,1981-02-27,EFETIVO,SECRETARIA MUNICIPAL DE CULTURA,BIBLIOTECA PUBLICA MUNICIPAL ERICO VERISSIMO,M,1951,True,78,48,True,73,43,True


In [22]:
## A partir do dataframe df_aposen gerado na função anterior, vamos analisar o quadro de servidores em cada setor, para os anos de 2024 e 2029. 
# Criaremos uma função que vai ler o dataset, e o ano de projeção (2024 OU 2029), e retornar uma tabela com 4 colunas: 'SETOR', 'TOTAL_SERV',  'SALDO_{ano_projecao}' , 'SALDO_ANALIS_{ano_projecao}'.
#  Na coluna 'SETOR', teremos o nome do  setor, na coluna 'TOTAL_SERV_2024', a contagem de servidores daquele setor no ano de 20204 , na coluna 'SALDO_2024' a quantidade da coluna 'TOTAL_SERV' - 'APTO_APOSENTADORIA_2024',  para o setor, e na coluna  'SALDO_ANALIS_2024' quero que seja o saldo, mas só de servidores que são analistas. Farei o mesmo pra 2029

#TODO: Consertar a função analise_quadro_servidores

def analise_quadro_servidores(df, projecao_ano):
    # Verificar se o ano de projeção é 2024 ou 2029
    if projecao_ano not in [2024, 2029]:
        raise ValueError("Ano de projeção deve ser 2024 ou 2029")

    # Primeiro criaremos uma tabela de contagem de servidores por setor
    setores_contagem = df['SETOR'].value_counts().reset_index()
    setores_contagem.columns = ['SETOR', 'TOTAL_SERV']

    # Agora vamos calcular o saldo de servidores restantes após as aposentadorias
    df_apto = df[df[f'APTO_APOSENTADORIA_{projecao_ano}'] == True]
    setores_apto = df_apto['SETOR'].value_counts().reset_index()
    setores_apto.columns = ['SETOR', 'APTO_APOSENTADORIA']
    setores_contagem = setores_contagem.merge(setores_apto, on='SETOR', how='left')
    
    # Preencher missing values com zero e converter para int
    setores_contagem[f'APTO_APOSENTADORIA_{projecao_ano}'] = setores_contagem['APTO_APOSENTADORIA'].fillna(0).astype(int)
    
    setores_contagem[f'SALDO_{projecao_ano}'] = setores_contagem['TOTAL_SERV'] - setores_contagem[f'APTO_APOSENTADORIA_{projecao_ano}']

    # Agora vamos calcular o saldo de servidores analistas restantes após as aposentadorias
    df_analista = df[df['CARGO_BASICO'].str.contains('ANALISTA DE INFORMACOES CULTURA E DESPORTO', na=False)]
    df_analista_apto = df_analista[df_analista[f'APTO_APOSENTADORIA_{projecao_ano}'] == True]
    setores_analista_apto = df_analista_apto['SETOR'].value_counts().reset_index()
    setores_analista_apto.columns = ['SETOR', f'SALDO_ANALIS_{projecao_ano}']
    setores_contagem = setores_contagem.merge(setores_analista_apto, on='SETOR', how='left')
    
    # Preencher missing values com zero e converter para int
    setores_contagem[f'SALDO_ANALIS_{projecao_ano}'] = setores_contagem[f'SALDO_ANALIS_{projecao_ano}'].fillna(0).astype(int)
    setores_contagem[f'SALDO_{projecao_ano}'] = setores_contagem[f'SALDO_{projecao_ano}'].astype(int)
    setores_contagem['TOTAL_SERV'] = setores_contagem['TOTAL_SERV'].astype(int)

    return setores_contagem


df_quadro_servidores_2024 = analise_quadro_servidores(df_aposen, 2024)
df_quadro_servidores_2029 = analise_quadro_servidores(df_aposen, 2029)



In [ ]:
## Criaremos uma tabela para facilitar a visualização da variação do quadro de servidores, nosso objetivo é identificar como ficará a quantidade de servidores em cada setor, caso os servidores aptos a se aposentar, de fato se aposentem.

def criar_tabela_aposentadoria(df, ano_projecao):
    # Verificar se o ano de projeção é 2024 ou 2029
    if ano_projecao not in [2024, 2029]:
        raise ValueError("Ano de projeção deve ser 2024 ou 2029")

    # Contar o número total de servidores por setor
    total_servidores_por_setor = df['SETOR'].value_counts().reset_index()
    total_servidores_por_setor.columns = ['SETOR', 'TOTAL_SERV']

    # Contar o número de servidores aptos a se aposentar no ano de projeção por setor
    apto_aposentadoria_col = f'APTO_APOSENTADORIA_{ano_projecao}'
    aposentadoria = df[df[apto_aposentadoria_col] == True]['SETOR'].value_counts().reset_index()
    aposentadoria.columns = ['SETOR', f'APOSENTADORIA_{ano_projecao}']

    # Contar o número de analistas aptos a se aposentar no ano de projeção por setor
    analista_aposentadoria = df[(df[apto_aposentadoria_col] == True) & 
                                (df['CARGO_BASICO'].str.contains('ANALISTA'))]['SETOR'].value_counts().reset_index()
    analista_aposentadoria.columns = ['SETOR', f'APOSENTADORIA_ANALISTA_{ano_projecao}']

    # Mesclar as contagens em um único DataFrame
    tabela = total_servidores_por_setor.merge(aposentadoria, on='SETOR', how='left').merge(analista_aposentadoria, on='SETOR', how='left')

    # Preencher valores NaN com 0
    tabela = tabela.fillna(0)

    # Calcular o saldo de servidores após aposentadorias
    tabela[f'SALDO_{ano_projecao}'] = tabela['TOTAL_SERV'] - tabela[f'APOSENTADORIA_{ano_projecao}']
    tabela[f'SALDO_ANALIS_{ano_projecao}'] = tabela['TOTAL_SERV'] - tabela[f'APOSENTADORIA_ANALISTA_{ano_projecao}']

    # Selecionar as colunas finais
    tabela_final = tabela[['SETOR', 'TOTAL_SERV', f'SALDO_{ano_projecao}', f'SALDO_ANALIS_{ano_projecao}']]

    return tabela_final

contagem_servidores_2024 = criar_tabela_aposentadoria(df_apt_apos_2024, 2024)

## Vamos criar uma nova visualização a partir da tabela 


In [7]:
def visualizar_variacao_aposentadorias(df_apt_apos_2024, df_apt_apos_2029):
    ## Agrupar os dados por setor e contar a quantidade de servidores aptos a se aposentar em 2024 e 2029
    df_apt_apos_2024 = df_apt_apos_2024.groupby('SETOR')['APTO_APOSENTADORIA'].sum().reset_index()
    df_apt_apos_2029 = df_apt_apos_2029.groupby('SETOR')['APTO_APOSENTADORIA'].sum().reset_index()

    ## Renomear as colunas
    df_apt_apos_2024.columns = ['SETOR', 'APTO_APOSENTAR_2024']
    df_apt_apos_2029.columns = ['SETOR', 'APTO_APOSENTAR_2029']

    ## Juntar os dados
    df_variacao = pd.merge(df_apt_apos_2024, df_apt_apos_2029, on='SETOR', how='outer')

    ## Preencher os valores nulos com 0
    df_variacao = df_variacao.fillna(0)

    ## Visualizar o resultado
    print(df_variacao)

# visualizar_variacao_aposentadorias(df_apt_apos_2024, df_apt_apos_2029)


NameError: name 'df_apt_apos_2024' is not defined

In [ ]:
##Mostrar as informações dos setores do ano de 2024 e 2019
def contar_servidores_por_setor(df, ano):
    setores_contagem = df['SETOR'].value_counts().reset_index()
    setores_contagem.columns = ['SETOR', 'QUANTIDADE']
    setores_contagem = setores_contagem.sort_values(by='QUANTIDADE', ascending=False)
    print(f'Quantidade de servidores por setor em {ano}:\n{setores_contagem}')
    return setores_contagem

setores_contagem_24 = contar_servidores_por_setor(df_bibliotecas24, 2024)
setores_contagem_19 = contar_servidores_por_setor(df_bibliotecas19, 2019)

df_bibliotecas24








In [ ]:
##  Criaremos uma tabela comparativa entre os dois anos, mostrando a quantidade de servidores em cada setor em 2019 e 2024
comparativo = pd.merge(setores_contagem_19, setores_contagem_24, on='SETOR', how='outer')
comparativo.columns = ['SETOR', '2019', '2024']
comparativo = comparativo.fillna(0)
comparativo['2019'] = comparativo['2019'].astype(int)
comparativo['2024'] = comparativo['2024'].astype(int)
comparativo = comparativo.sort_values(by='2024', ascending=False)
# print(f'Comparativo entre 2019 e 2024:\n{comparativo}')

## Queremos uma tabela com o total de servidores em 2019 e 2024
total_2019 = setores_contagem_19['QUANTIDADE'].sum()
total_2024 = setores_contagem_24['QUANTIDADE'].sum()
# print(f'Total de servidores em 2019: {total_2019}')
# print(f'Total de servidores em 2024: {total_2024}')


## Criaremos uma função que gerará uma tabela com as colunas SETOR, QUANTIDADE_SERVIDORES, e QUANTIDADE_APTOS_A_APOSENTAR

def comparativo_aposentadoria(df_aposentadoria, setores_contagem_24):
    ## Criar uma nova tabela com o nome do setor, a quantidade de servidores em 2024 e a quantidade de servidores aptos a se aposentar em 2024
    df_aposentadoria_por_setor = pd.merge(setores_contagem_24, df_aposentadoria['SETOR'].value_counts().reset_index(), on='SETOR', how='left')
    df_aposentadoria_por_setor.columns = ['SETOR', 'QUANTIDADE_SERVIDORES', 'QUANTIDADE_APTOS_A_APOSENTAR']
    df_aposentadoria_por_setor = df_aposentadoria_por_setor.fillna(0)

    # Criar uma coluna com a subtração entre a quantidade de servidores e a quantidade de servidores aptos a se aposentar
    df_aposentadoria_por_setor['QUANTIDADE_NAO_APTOS_A_APOSENTAR'] = df_aposentadoria_por_setor['QUANTIDADE_SERVIDORES'] - df_aposentadoria_por_setor['QUANTIDADE_APTOS_A_APOSENTAR']
    
    # Transformar os valores float em int
    df_aposentadoria_por_setor['QUANTIDADE_SERVIDORES'] = df_aposentadoria_por_setor['QUANTIDADE_SERVIDORES'].astype(int)
    df_aposentadoria_por_setor['QUANTIDADE_APTOS_A_APOSENTAR'] = df_aposentadoria_por_setor['QUANTIDADE_APTOS_A_APOSENTAR'].astype(int)
    df_aposentadoria_por_setor['QUANTIDADE_NAO_APTOS_A_APOSENTAR'] = df_aposentadoria_por_setor['QUANTIDADE_NAO_APTOS_A_APOSENTAR'].astype(int)

    return df_aposentadoria_por_setor


df_aposentadoria_por_setor = comparativo_aposentadoria(df_aposentadoria, setores_contagem_24)

## Criaremos uma função que filtrará os servidores pelo valor da coluna CARGO_BASICO, retornando aqueles que contém a substring 'ANALISTA DE INFORMACOES CULTURA E DESPORTO' no nome do cargo, desses servidores, retornaremos os que possuem a coluna 'APTA_A_APOSENTAR' como False

def filtrar_analista(df_bibliotecas24_resolvido):
    ## Filtrar os servidores pelo valor da coluna CARGO_BASICO
    df_analista = df_bibliotecas24_resolvido[df_bibliotecas24_resolvido['CARGO_BASICO'].str.contains('ANALISTA DE INFORMACOES CULTURA E DESPORTO')]
    ## Filtrar os servidores que possuem a coluna 'APTA_A_APOSENTAR' como False
    df_analista_nao_apto = df_analista[df_analista['APTA_A_APOSENTAR'] == False]
    return df_analista_nao_apto

df_analista_nao_apto = filtrar_analista(df_bibliotecas24_resolvido)

##Criaremos uma função que gerará uma tabela com as colunas SETOR, QUANTIDADE_SERVIDORES, e QUANTIDADE_NAO_APTOS_A_APOSENTAR, considerando apenas os servidores analistas

def comparativo_aposentadoria_analista(df_analista_nao_apto):
    ## Criar uma nova tabela com o nome do setor, a quantidade de servidores em 2024 e a quantidade de servidores não aptos a se aposentar em 2024, se o número for zero, preencheremos com zero
    df_analista_nao_apto_por_setor = df_analista_nao_apto['SETOR'].value_counts().reset_index()
    df_analista_nao_apto_por_setor.columns = ['SETOR', 'QUANTIDADE_NAO_APTOS_A_APOSENTAR']
    df_analista_nao_apto_por_setor = pd.merge(setores_contagem_24, df_analista_nao_apto_por_setor, on='SETOR', how='left')
    df_analista_nao_apto_por_setor = df_analista_nao_apto_por_setor.fillna(0)
    df_analista_nao_apto_por_setor['QUANTIDADE_NAO_APTOS_A_APOSENTAR'] = df_analista_nao_apto_por_setor['QUANTIDADE_NAO_APTOS_A_APOSENTAR'].astype(int)
    
    

    return df_analista_nao_apto_por_setor

df_analista_nao_apto_por_setor = comparativo_aposentadoria_analista(df_analista_nao_apto)
df_analista_nao_apto_por_setor.head(10)

In [ ]:
## Vamos visualizar a distribuição espacial desses servidores
%pip install geopandas
%pip install matplotlib

import matplotlib.pyplot as plt
import geopandas as gpd

## Carregar o arquivo shapefile
mapa_setores = gpd.read_file(r'C:\Repos\tcc-bruna\DEINFO_DISTRITO.shp')

endereco_setores = []


